In [1]:
from astropy.coordinates import SkyCoord, ICRS, CartesianRepresentation, CartesianDifferential, Galactic, Galactocentric
import astropy.units as u
from astropy.io import fits
%pylab inline
import ebf
import shutil
import subprocess
import os, sys
path = os.path.abspath('../library/')
if path not in sys.path:
    sys.path.append(path)
from convert_to_recarray import create_gdr2mock_mag_limited_survey_from_nbody

Populating the interactive namespace from numpy and matplotlib


In [2]:
"""
Plummer model generator
This module contains a function used to create Plummer (1911) models, which 
follow a spherically symmetric density profile of the form:
rho = c * (1 + r**2)**(-5/2)
"""

import numpy
import numpy.random

from math import pi, sqrt

from amuse.units import nbody_system
from amuse import datamodel

__all__ = ["new_plummer_sphere", "new_plummer_model"]

class MakePlummerModel(object):
    def __init__(self, number_of_particles, convert_nbody = None, radius_cutoff = 22.8042468, mass_cutoff = 0.999,
            do_scale = False, random_state = None, random = None):
        self.number_of_particles = number_of_particles
        self.convert_nbody = convert_nbody
        self.mass_cutoff = min(mass_cutoff, self.calculate_mass_cuttof_from_radius_cutoff(radius_cutoff))
        self.do_scale = do_scale
        if not random_state == None:
            print("DO NOT USE RANDOM STATE")
        
        self.random_state = None
        
        if random is None:
            self.random = numpy.random
        else:
            self.random = random

    def calculate_mass_cuttof_from_radius_cutoff(self, radius_cutoff):
        if radius_cutoff > 99999:
            return 1.0
        scale_factor = 16.0 / (3.0 * pi)
        rfrac = radius_cutoff * scale_factor
        denominator = pow(1.0 + rfrac ** 2, 1.5)
        numerator = rfrac ** 3
        return numerator/denominator

    def calculate_radius(self, index):
        mass_min = (index * self.mass_cutoff) / self.number_of_particles
        mass_max = ((index+1) * self.mass_cutoff) / self.number_of_particles
        random_mass_fraction = self.random.uniform(mass_min, mass_max)
        radius = 1.0 / sqrt( pow (random_mass_fraction, -2.0/3.0) - 1.0)
        return radius

    def calculate_radius_uniform_distribution(self):
        return 1.0 /  numpy.sqrt( numpy.power(self.random.uniform(0,self.mass_cutoff,(self.number_of_particles,1)), -2.0/3.0) - 1.0)

    def new_positions_spherical_coordinates(self):
        pi2 = pi * 2
        radius = self.calculate_radius_uniform_distribution()
        theta = numpy.arccos(self.random.uniform(-1.0,1.0, (self.number_of_particles,1)))
        phi = self.random.uniform(0.0,pi2, (self.number_of_particles,1))
        return (radius,theta,phi)

    def new_velocities_spherical_coordinates(self, radius):
        pi2 = pi * 2
        x,y = self.new_xy_for_velocity()
        velocity = x * sqrt(2.0) * numpy.power( 1.0 + radius*radius, -0.25)
        theta = numpy.arccos(self.random.uniform(-1.0,1.0, (self.number_of_particles,1)))
        phi = self.random.uniform(0.0,pi2, (self.number_of_particles,1))
        return (velocity,theta,phi)

    def coordinates_from_spherical(self, radius, theta, phi):
        x = radius * numpy.sin( theta ) * numpy.cos( phi )
        y = radius * numpy.sin( theta ) * numpy.sin( phi )
        z = radius * numpy.cos( theta )
        return (x,y,z)

    def new_xy_for_velocity(self):
        number_of_selected_items = 0
        selected_values_for_x = numpy.zeros(0)
        selected_values_for_y = numpy.zeros(0)
        while (number_of_selected_items < self.number_of_particles):
            x = self.random.uniform(0,1.0, (self.number_of_particles-number_of_selected_items))
            y = self.random.uniform(0,0.1, (self.number_of_particles-number_of_selected_items))
            g = (x**2) * numpy.power(1.0 - x**2, 3.5)
            compare = y <= g
            selected_values_for_x = numpy.concatenate((selected_values_for_x, x.compress(compare)))
            selected_values_for_y= numpy.concatenate((selected_values_for_x, y.compress(compare)))
            number_of_selected_items = len(selected_values_for_x)
        return numpy.atleast_2d(selected_values_for_x).transpose(), numpy.atleast_2d(selected_values_for_y).transpose()

    def new_model(self):
        m = numpy.zeros((self.number_of_particles,1)) + (1.0 / self.number_of_particles)
        radius, theta, phi = self.new_positions_spherical_coordinates()
        position =  numpy.hstack(self.coordinates_from_spherical(radius, theta, phi))
        radius, theta, phi = self.new_velocities_spherical_coordinates(radius)
        velocity = numpy.hstack(self.coordinates_from_spherical(radius, theta, phi))
        position = position / 1.695
        velocity = velocity / sqrt(1 / 1.695)
        return (m, position, velocity)

    @property
    def result(self):
        masses = numpy.ones(self.number_of_particles) / self.number_of_particles
        radius, theta, phi = self.new_positions_spherical_coordinates()
        x,y,z =  self.coordinates_from_spherical(radius, theta, phi)
        radius, theta, phi = self.new_velocities_spherical_coordinates(radius)
        vx,vy,vz = self.coordinates_from_spherical(radius, theta, phi)
 
        result = datamodel.Particles(self.number_of_particles)
        result.mass = nbody_system.mass.new_quantity(masses)
        result.x = nbody_system.length.new_quantity(x.reshape(self.number_of_particles)/1.695)
        result.y = nbody_system.length.new_quantity(y.reshape(self.number_of_particles)/1.695)
        result.z = nbody_system.length.new_quantity(z.reshape(self.number_of_particles)/1.695)
        result.vx = nbody_system.speed.new_quantity(vx.reshape(self.number_of_particles) / sqrt(1/1.695))
        result.vy = nbody_system.speed.new_quantity(vy.reshape(self.number_of_particles) / sqrt(1/1.695))
        result.vz = nbody_system.speed.new_quantity(vz.reshape(self.number_of_particles) / sqrt(1/1.695))
        result.radius = 0 | nbody_system.length

        result.move_to_center()
        if self.do_scale:
            result.scale_to_standard()

        if not self.convert_nbody is None:
            result = datamodel.ParticlesWithUnitsConverted(result, self.convert_nbody.as_converter_from_si_to_generic())
            result = result.copy()

        return result

def new_plummer_model(number_of_particles, *list_arguments, **keyword_arguments):
    """
    Create a plummer sphere with the given number of particles. Returns
    a set of stars with equal mass and positions and velocities distributed
    to fit a plummer star distribution model. The model is centered around the
    origin. Positions and velocities are optionally scaled such that the kinetic and
    potential energies are 0.25 and -0.5 in nbody-units, respectively.
    :argument number_of_particles: Number of particles to include in the plummer sphere
    :argument convert_nbody:  When given will convert the resulting set to SI units
    :argument radius_cutoff: Cutoff value for the radius (defaults to 22.8042468)
    :argument mass_cutoff: Mass percentage inside radius of 1
    :argument do_scale: scale the result to exact nbody units (M=1, K=0.25, U=-0.5)
    """
    uc = MakePlummerModel(number_of_particles, *list_arguments, **keyword_arguments)
    return uc.result

new_plummer_sphere = new_plummer_model

In [3]:
i=0
cat = fits.getdata('../input/fake_cluster/cluster_for_processing.fits')
for item in cat.dtype.names:
    print(item,cat[item][i])
#cat = cat[:1]

clusterName Lynga_15
ra 175.523
dec -62.457
r50 3.6273161956
pmra -6.477
pmdec 0.793
age_gyr 0.0251188643151
FeH_synth 0.0499636163811
rvs 0.0166786820078
mass 301.274217933
vrot 0.100198746965
distance_pc 1717.6
x0 -0.438507686583
y0 0.109971617499
z0 0.891973795665


In [4]:
nbody_folder = '/home/rybizki/Programme/GalaxiaData/'
folder = 'cluster/'
filename = 'cluster_list'
folder_cat = '../output/Clusters'

# Need to specify where the GalaxiaData folder is and how to name the new simulation
names = cat.clusterName

# 2 input files for Galaxia need to be created
folder_create = nbody_folder + 'nbody1/' + folder
if os.path.exists(folder_create):
    shutil.rmtree(folder_create)
    os.mkdir(folder_create)
    print(folder_create, "existed and was recreated")
else:
    os.mkdir(folder_create)


# Here the file which tells Galaxia where to find the input file is created
filedata = 'nbody1/%s\n         %d       1\n' %(folder,len(names))
for item in names:
    filedata += '%s%s.ebf\n' %(filename,item.decode())
#filedata = 'nbody1/%s\n         %d       1\n%s.ebf\n' %(folder,1, filename + names[1])
file = open(nbody_folder + "nbody1/filenames/" + filename + ".txt", "w")
file.write(filedata)
file.close()



/home/rybizki/Programme/GalaxiaData/nbody1/cluster/ existed and was recreated


In [5]:
import numpy as np
from amuse.units import units

In [6]:
for i in range(len(cat)):
    print(i,names[i].decode(), len(cat))
    ra = cat.ra[i]
    dec = cat.dec[i]
    distance = cat.distance_pc[i]
    pmra = cat.pmra[i]
    pmdec = cat.pmdec[i]
    rvs = cat.rvs[i]
    age = cat.age_gyr[i]
    feh = cat.FeH_synth[i]
    mass = cat.mass[i]
    name = cat.clusterName[i].decode()
    #Galactocentric coordinate system
    c = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, distance=(distance/1000.)*u.kpc, frame='icrs',
                 pm_ra_cosdec = pmra*u.mas/u.yr, pm_dec = pmdec*u.mas/u.yr, radial_velocity = rvs*u.km/u.s,
                 galcen_distance = 8.0*u.kpc, z_sun = 15.0*u.pc,
                 galcen_v_sun=CartesianDifferential(d_x=11.1*u.km/u.s, d_y=239.08*u.km/u.s, d_z=7.25*u.km/u.s))
    pos_x = c.galactocentric.x.value
    pos_y = c.galactocentric.y.value
    pos_z = c.galactocentric.z.value
    vel_x = c.galactocentric.v_x.value
    vel_y = c.galactocentric.v_y.value
    vel_z = c.galactocentric.v_z.value
    # setting the physical lengthscale
    l_scale = cat.r50[i]
    # random spin axis vector
    x0 = cat.x0[i]
    y0 = cat.y0[i]
    z0 = cat.z0[i]
    #Generating nbody particles from cluster information
    Mcluster = mass | units.MSun
    Rcluster= l_scale | units.parsec
    converter= nbody_system.nbody_to_si(Mcluster,Rcluster)
    nparticles = 1000
    stars = new_plummer_sphere(nparticles,converter)

    nbx =stars.x.value_in(units.pc)
    nby =stars.y.value_in(units.pc)
    nbz =stars.z.value_in(units.pc)
    nbmass = stars.mass.value_in(units.MSun)
    nbvx = stars.vx.value_in(units.kms)
    nbvy = stars.vy.value_in(units.kms)
    nbvz = stars.vz.value_in(units.kms)
    
    #distance_to_rot_axis = |x0_vec x nbx_vec| (for line through origin and spin axis being normed)
    resx = y0*nbz - z0*nby
    resy = z0*nbx - x0*nbz
    resz = x0*nby - y0*nbx
    # this then scales the rotational velocity
    dist_spin_axis = np.sqrt(resx**2+resy**2+resz**2)
    vrot = np.divide(l_scale,dist_spin_axis)*cat.vrot[i]
    # direction of the rotation (perpendicular to spin axis and the point connected to origin)
    normx = np.divide(resx,dist_spin_axis)
    normy = np.divide(resy,dist_spin_axis)
    normz = np.divide(resz,dist_spin_axis)
    # speed xyz
    speed_x = normx*vrot
    speed_y = normy*vrot
    speed_z = normz*vrot
    # rescale nbxyz vectors to kpc and add galactocentric xyz
    nbx = np.divide(nbx,1000) + pos_x
    nby = np.divide(nby,1000) + pos_y
    nbz = np.divide(nbz,1000) + pos_z
    nbspeed_x = speed_x + vel_x + nbvx
    nbspeed_y = speed_y + vel_y + nbvy
    nbspeed_z = speed_z + vel_z + nbvz
    # galaxia format
    pos = np.zeros((nparticles,3))
    vel = np.zeros((nparticles,3))
    pos[:,0] = nbx
    pos[:,1] = nby
    pos[:,2] = nbz
    vel[:,0] = nbspeed_x
    vel[:,1] = nbspeed_y
    vel[:,2] = nbspeed_z
    # assign age
    nbage = np.linspace(age-0.0009,age+0.0009,num = nparticles)
    nbfeh = np.sort(np.random.normal(feh,0.01,nparticles))
    nbage = nbage[::-1] # check if that is necessary and results in the right age
    nbalpha = np.zeros(nparticles)


    # writing to ebf file for galaxia
    ebf.write(nbody_folder + 'nbody1/' + folder + filename + name + '.ebf', '/mass', nbmass,'w')	
    ebf.write(nbody_folder + 'nbody1/' + folder + filename + name + '.ebf', '/feh', nbfeh,'a')
    ebf.write(nbody_folder + 'nbody1/' + folder + filename + name + '.ebf', '/id', 1,'a')
    ebf.write(nbody_folder + 'nbody1/' + folder + filename + name + '.ebf', '/alpha', nbalpha,'a')
    ebf.write(nbody_folder + 'nbody1/' + folder + filename + name + '.ebf', '/age', nbage,'a')
    ebf.write(nbody_folder + 'nbody1/' + folder + filename + name + '.ebf', '/pos3', pos,'a')
    ebf.write(nbody_folder + 'nbody1/' + folder + filename + name + '.ebf', '/vel3', vel,'a')

    # Preparing file for Enbid
    ps = np.concatenate((pos,vel),axis = 1)
    enbid_filename = '%s.dat' %(name)
    np.savetxt(enbid_filename,ps,fmt='%.6f')
    # Making the parameterfile for Enbid
    filedata = 'InitCondFile     %s\nICFormat                  0     \nSnapshotFileBase        _ph3\nSpatialScale            1 \nPartBoundary            7   \nNodeSplittingCriterion  1   \nCubicCells              1   \nMedianSplittingOn       0   \nTypeOfSmoothing      3\nDesNumNgb            64   \nVolCorr              1   \nTypeOfKernel           3 \nKernelBiasCorrection   1    \nAnisotropicKernel      0   \nAnisotropy             0   \nDesNumNgbA             128 \nTypeListOn        0\nPeriodicBoundaryOn 0 \n\n' %(enbid_filename)
    myparameterfile = "myparameterfile3"
    file = open(myparameterfile, "w")
    file.write(filedata)
    file.close()
    #Running Enbid
    args = ['./Enbid', myparameterfile]
    p = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)	
    #print("Enbid calculates smoothing length")
    (output, err) = p.communicate()
    # Writing to nbody smoothing length file 
    t = np.genfromtxt(enbid_filename + "_ph3.est",skip_header=1)
    d6 = np.zeros((len(pos),2))
    d6[:,0] = t[:,1]
    d6[:,1] = t[:,2]
    #print(d6.shape,t.shape)
    ebf.write(nbody_folder + 'nbody1/' + folder + filename + name + '_d6n64_den.ebf', '/h_cubic', d6, 'w')
    # remove temporary files
    os.remove(enbid_filename)
    os.remove(enbid_filename + "_ph3.est")
    # Same for 3d
    # Preparing file for Enbid
    enbid_filename = '%s3d.dat' %(name)
    np.savetxt(enbid_filename,pos,fmt='%.6f')
    # Making the parameterfile for Enbid
    filedata = 'InitCondFile     %s\nICFormat                  0     \nSnapshotFileBase        _ph3\nSpatialScale            1 \nPartBoundary            7   \nNodeSplittingCriterion  1   \nCubicCells              1   \nMedianSplittingOn       0   \nTypeOfSmoothing      3\nDesNumNgb            64   \nVolCorr              1   \nTypeOfKernel           3 \nKernelBiasCorrection   1    \nAnisotropicKernel      0   \nAnisotropy             0   \nDesNumNgbA             128 \nTypeListOn        0\nPeriodicBoundaryOn 0 \n\n' %(enbid_filename)
    myparameterfile = "myparameterfile3"
    file = open(myparameterfile, "w")
    file.write(filedata)
    file.close()
    #Running Enbid
    args = ['./Enbid3d', myparameterfile]
    p = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)	
    #print("Enbid calculates smoothing length")
    (output, err) = p.communicate()
    # Writing to nbody smoothing length file 
    t = np.genfromtxt(enbid_filename + "_ph3.est",skip_header=1)
    d3 = np.zeros((len(pos)))
    d3 = t[:,1]
    #print(d3.shape,t.shape)
    ebf.write(nbody_folder + 'nbody1/' + folder + filename + name + '_d3n64_den.ebf', '/h_cubic', d3, 'w')
    # remove temporary files
    os.remove(enbid_filename)
    os.remove(enbid_filename + "_ph3.est")



0 Lynga_15 1118
1 FSR_1051 1118
2 FSR_1397 1118
3 Ruprecht_43 1118
4 SAI_72 1118
5 ESO_313_03 1118
6 Czernik_26 1118
7 Berkeley_14A 1118
8 Stock_17 1118
9 DBSB_21 1118
10 FSR_1025 1118
11 FSR_1580 1118
12 Ruprecht_24 1118
13 Teutsch_42 1118
14 Berkeley_29 1118
15 FSR_0401 1118
16 FSR_0975 1118
17 FSR_1419 1118
18 Koposov_43 1118
19 FSR_1172 1118
20 Ivanov_8 1118
21 Alessi_15 1118
22 Alessi_59 1118
23 Basel_10 1118
24 ESO_559_13 1118
25 FSR_1460 1118
26 Ruprecht_32 1118
27 FSR_0284 1118
28 FSR_0524 1118
29 FSR_1335 1118
30 Hogg_10 1118
31 NGC_1624 1118
32 SAI_47 1118
33 Teutsch_13 1118
34 Teutsch_52 1118
35 Turner_3 1118
36 Kronberger_54 1118
37 Stock_13 1118
38 Teutsch_50 1118
39 Czernik_43 1118
40 FSR_1171 1118
41 Graham_1 1118
42 Mamajek_1 1118
43 Schuster_1 1118
44 Teutsch_27 1118
45 FSR_1032 1118
46 Ivanov_4 1118
47 NGC_1444 1118
48 Platais_10 1118
49 Teutsch_31 1118
50 ASCC_66 1118
51 Berkeley_102 1118
52 ESO_393_15 1118
53 FSR_0977 1118
54 FSR_1352 1118
55 Ruprecht_77 1118
56 DB2

436 Haffner_4 1118
437 King_12 1118
438 NGC_5269 1118
439 vdBergh_1 1118
440 Alessi_19 1118
441 FSR_0534 1118
442 King_15 1118
443 NGC_2183 1118
444 Teutsch_10 1118
445 ASCC_101 1118
446 NGC_1901 1118
447 Ruprecht_84 1118
448 BH_84 1118
449 Juchert_Saloran_1 1118
450 Loden_372 1118
451 Haffner_23 1118
452 NGC_637 1118
453 NGC_6520 1118
454 NGC_7296 1118
455 Pismis_23 1118
456 Ruprecht_48 1118
457 SAI_81 1118
458 Berkeley_71 1118
459 Kronberger_4 1118
460 NGC_6031 1118
461 NGC_6561 1118
462 Ruprecht_127 1118
463 Teutsch_144 1118
464 BDSB96 1118
465 Berkeley_6 1118
466 IC_4996 1118
467 Lynga_6 1118
468 NGC_2374 1118
469 SAI_113 1118
470 vdBergh_80 1118
471 Alessi_60 1118
472 BH_55 1118
473 NGC_136 1118
474 NGC_2129 1118
475 NGC_5749 1118
476 BH_85 1118
477 FSR_0166 1118
478 FSR_0306 1118
479 FSR_1591 1118
480 Ruprecht_26 1118
481 Teutsch_145 1118
482 Aveni_Hunter_1 1118
483 BH_144 1118
484 King_20 1118
485 Ruprecht_44 1118
486 FSR_1342 1118
487 Koposov_36 1118
488 NGC_744 1118
489 Ruprec

868 NGC_2670 1118
869 Haffner_13 1118
870 NGC_4349 1118
871 Platais_8 1118
872 Tombaugh_1 1118
873 Ruprecht_91 1118
874 Collinder_115 1118
875 NGC_1193 1118
876 NGC_1798 1118
877 Berkeley_43 1118
878 NGC_2353 1118
879 NGC_6531 1118
880 NGC_2669 1118
881 Trumpler_13 1118
882 Trumpler_3 1118
883 BH_121 1118
884 NGC_6204 1118
885 IC_2391 1118
886 NGC_2309 1118
887 ASCC_16 1118
888 NGC_5999 1118
889 King_10 1118
890 Dias_6 1118
891 Czernik_37 1118
892 Ruprecht_145 1118
893 Ruprecht_18 1118
894 ASCC_108 1118
895 NGC_6728 1118
896 FSR_0342 1118
897 NGC_4337 1118
898 NGC_2547 1118
899 NGC_6451 1118
900 Czernik_41 1118
901 King_6 1118
902 Cep_OB5 1118
903 NGC_6664 1118
904 NGC_1662 1118
905 NGC_609 1118
906 Berkeley_59 1118
907 NGC_752 1118
908 NGC_6167 1118
909 Collinder_197 1118
910 Berkeley_24 1118
911 Berkeley_68 1118
912 NGC_2266 1118
913 Roslund_6 1118
914 FSR_0088 1118
915 NGC_6087 1118
916 Berkeley_8 1118
917 NGC_2262 1118
918 NGC_2509 1118
919 NGC_5662 1118
920 NGC_2425 1118
921 ASCC_

In [8]:
seed = 1
create_gdr2mock_mag_limited_survey_from_nbody(nbody_filename = filename,nside = 512,
                                              outputDir = folder_cat + "_%d" %(seed),
                                   use_previous = False, delete_ebf = True,
                                  fSample = 1, make_likelihood_asessment=False, seed = seed, popid=11)

/home/rybizki/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/rybizki/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/rybizki/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


../output/Clusters_1/ existed and was recreated
Galaxia spawns catalogue
output:  b'Galaxia-v0.81\nCODEDATAPATH=/home/rybizki/Programme/GalaxiaData/\nReading Parameter file-             ../output/Clusters_1/nbody.log\n--------------------------------------------------------\noutputFile               nbody                   \nmodelFile                Model/population_parameters_BGM_update.ebf\ncodeDataDir              /home/rybizki/Programme/GalaxiaData\noutputDir                ../output/Clusters_1    \nphotoSys                 parsec1/GAIADR3         \nmagcolorNames            gaia_g,gaia_bpft-gaia_rp\nappMagLimits[0]          -1000.000000            \nappMagLimits[1]          20.700000               \nabsMagLimits[0]          -1000.000000            \nabsMagLimits[1]          1000.000000             \ncolorLimits[0]           -1000.000000            \ncolorLimits[1]           1000.000000             \ngeometryOption           0                       \nlongitude                0.00000

541004
('rad', 'teff', 'vx', 'vy', 'vz', 'pz', 'px', 'py', 'feh', 'exbv_schlegel', 'lum', 'glon', 'glat', 'smass', 'age', 'grav', 'gaia_g', 'gaia_bpft', 'gaia_bpbr', 'gaia_rp', 'gaia_rvs', 'popid', 'mact')
converting to npy and appending ra and dec took 2.8 sec
0 541004
converting time and applying extinction map for 541004 sources in nside = 512 took 2.3 sec
indexing and remapping to isochrones took 5.6 sec
calculating extinction curve for all bands took 13.9 sec
541004
441697
calculated healpix
calculated pmdec pmra and rv
cleaning of data took 3.5 sec


/home/rybizki/Desktop/Galaxia_wrap-master/library/defaults.py:9: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/rybizki/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/rybizki/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/rybizki/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/rybizki/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/rybizki/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    iol

total number of stars = 441697
0.0
441697.0
33681395.7873
plotting time took 2.5 sec
Total time in minutes: 0.6
